Investigating how much of each city is within x min. walk of a station

Imports and useful constants

In [1]:
import geopandas
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import sys
sys.path.append("C:\\Users\\beezy\\git\\bluebikes_system_analysis\\code")
sys.path.append("C:\\Users\\beezy\\git\\bluebikes_system_analysis\\code\\data_handling")

# for loading station information
from data_handling.data_loading_methods import *

# useful crs constants
crs_lonlat = 'EPSG:4269'
crs_cartesian_feet = 'EPSG:2263'

In [2]:
# select walking time in minutes
walktime = 3 # minutes

In [3]:
# function to calculate area coverage by city

def calc_area_coverage(cityname, walktime):
    """
    Calculates how much area of a city is covered within <walktime> min. of a station
    
    Inputs:
        cityname: string, name of city to compare to
        walktime: float, walking time in minutes
        
    Outputs:
        percent_coverage: % coverage
        select_citylimits: geodata for selected city
        intersection_wcity: geodata for intersection data
    """
    
    # load city limits geodata
    citylimits = load_MA_citylimits()
    
    # load bluebike station geodata
    stationdata = load_station_info()
    # convert to a geodataframe
    station_gdf = geopandas.GeoDataFrame(
        stationdata, geometry=geopandas.points_from_xy(stationdata['lon'], stationdata['lat']), crs=crs_lonlat)
    
    # convert gdf CRS to cartesian
    citylimits = citylimits.to_crs(crs=crs_cartesian_feet)
    station_gdf = station_gdf.to_crs(crs=crs_cartesian_feet)

    # calc walking distance
    mph = 3.25
    fph = mph * 5280 # feet per hour
    fpm = fph / 60 # feet per minute
    walkradius = walktime * fpm
    
    # buffer
    station_gdf['walkable_region'] = station_gdf.buffer(walkradius)
    station_gdf = station_gdf.set_geometry('walkable_region')
    
    # combine all shapes into one
    station_gdf_dissolved = station_gdf.dissolve().to_crs(crs_cartesian_feet)
    
    # intersect with chosen city
    select_citylimits = citylimits.loc[ citylimits['TOWN'] == cityname ].reset_index()
    intersection_wcity = select_citylimits.intersection( station_gdf_dissolved )

    # calculate area
    intersection_area = intersection_wcity.area
    city_area = select_citylimits.area
    percent_coverage = 100 * intersection_area[0]/city_area[0]
    
    return (percent_coverage, select_citylimits, intersection_wcity)

In [4]:
# calculate what percentage of boston can access bluebikes by x min walk

(percent_coverage, select_citylimits, intersection_wcity) = calc_area_coverage('BOSTON', walktime)

print( 'Percent of Boston within ' + str(walktime) + ' min. walk of a station: ' + str(percent_coverage) )

Percent of Boston within 3 min. walk of a station: 30.64215179377151


In [5]:
# view the area covered by boston on a map
ax = intersection_wcity.explore()
select_citylimits.explore( m = ax, style_kwds={'fill':False, 'color':'red'} )

In [6]:
# calculate what percentage of cambridge can access bluebikes by x min walk
(percent_coverage, select_citylimits, intersection_wcity) = calc_area_coverage('CAMBRIDGE', walktime)

print( 'Percent of Cambridge within ' + str(walktime) + ' min. walk of a station: ' + str(percent_coverage) )

Percent of Cambridge within 3 min. walk of a station: 59.793441184212064


In [7]:
# view the area covered by cambridge on a map
ax = intersection_wcity.explore()
select_citylimits.explore( m = ax, style_kwds={'fill':False, 'color':'red'} )

In [8]:
# calculate what percentage of somerville can access bluebikes by x min walk

(percent_coverage, select_citylimits, intersection_wcity) = calc_area_coverage('SOMERVILLE', walktime)

print( 'Percent of Somerville within ' + str(walktime) + ' min. walk of a station: ' + str(percent_coverage) )

Percent of Somerville within 3 min. walk of a station: 52.44840578919175


In [9]:
# view the area covered by cambridge on a map
ax = intersection_wcity.explore()
select_citylimits.explore( m = ax, style_kwds={'fill':False, 'color':'red'} )

In [10]:
# calculate what percentage of watertown can access bluebikes by x min walk

(percent_coverage, select_citylimits, intersection_wcity) = calc_area_coverage('WATERTOWN', walktime)

print( 'Percent of Watertown within ' + str(walktime) + ' min. walk of a station: ' + str(percent_coverage) )

Percent of Watertown within 3 min. walk of a station: 13.057865333923882


In [11]:
# view the area covered by watertown on a map
ax = intersection_wcity.explore()
select_citylimits.explore( m = ax, style_kwds={'fill':False, 'color':'red'} )